In [5]:
import numpy as np
import json
import time
import os

'''
INPUT
    filename: string containing the json file to be saved;
    X_lc: Compressed 1D/2D frequency domain signal for each window size; left channel
    X_rc: Compressed 1D/2D frequency domain signal for each window size; right channel
    i_lc: Indices of the original frequencies that you decide to keep. Same dimensions as X_lc.
    i_rc: Indices of the original frequencies that you decide to keep. Same dimensions as X_rc.
OUTPUT
    filesize: Size of json file [Byte]
    
This is a helper funcion. You don't need to modify it.
'''

def save_to_disk(filename, X_lc, X_rc, i_lc, i_rc):
    if filename[-4:]!='.json':
        filename = filename + '.json'
    # Reshape 1D to 2D
    if X_lc.ndim==1:
        X_lc = np.reshape(X_lc,[len(X_lc),1])
        X_rc = np.reshape(X_rc,[len(X_rc),1])
    num_comp, num_win = np.shape(X_lc)
    c_lc = np.array(np.zeros((num_comp, num_win)), dtype='int16')
    c_rc = np.array(np.zeros((num_comp, num_win)), dtype='int16')
    max_lc = np.max(X_lc)
    min_lc = np.abs(np.min(X_lc))
    max_rc = np.max(X_rc)
    min_rc = np.abs(np.min(X_rc))
    scale_factor = np.max([max_lc, min_lc, max_rc, min_rc])
    bits = 16-np.ceil(np.log2(scale_factor))
    for i in range(0,num_win):
        c_lc[:,i] = np.array(X_lc[:,i]*pow(2,bits-1), dtype='int16')
        c_rc[:,i] = np.array(X_rc[:,i]*pow(2,bits-1), dtype='int16')
        
    datadict = {
        "bits": bits,
        "i_lc": i_lc.tolist(),
        "i_rc": i_rc.tolist(),
        "c_lc": c_lc.tolist(),
        "c_rc": c_rc.tolist(),
    }
    
    with open(filename, 'w') as f:
        json.dump(datadict, f, separators=(',', ':'))
        
    time.sleep(0.5)
    filesize = os.path.getsize(filename)
    return filesize